This is an exploratory analysis of animal complaints in Brisbane. There are a few interesting things I'd like to show - animal complaints per suburb of course, but animal complaints per square kilometre and per person (by pulling suburb info as well) will be equally interesting. Trends over time - which suburbs have improved or gotten worse over a few quarters. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [16]:
apr_to_jun_19 = pd.read_csv("/home/jack/AnimalC/cars-bis-open-data-animal-related-complaints-apr-jun-2019.csv")

In [17]:
apr_to_jun_18 = pd.read_csv("/home/jack/AnimalC/cars-bis-open-data-animal-related-complaints-apr-to-jun-2018.csv")

In [18]:
jan_to_mar_19 = pd.read_csv("/home/jack/AnimalC/cars-bis-open-data-animal-related-complaints-jan-mar-2019.csv")

In [19]:
jan_to_mar_18 = pd.read_csv("/home/jack/AnimalC/cars-bis-open-data-animal-related-complaints-jan-to-mar-2018.csv")

In [20]:
jul_to_sep_18 = pd.read_csv("/home/jack/AnimalC/cars-bis-open-data-animal-related-complaints-jul-to-sep-2018.csv")

In [21]:
jul_to_sep_19 = pd.read_csv("/home/jack/AnimalC/cars-bis-open-data-animal-related-complaints-jul-to-sep-2019.csv")

In [22]:
oct_to_dec_18 = pd.read_csv("/home/jack/AnimalC/cars-bis-open-data-animal-related-complaints-oct-to-dec-2018.csv")

In [23]:
oct_to_dec_19 = pd.read_csv("/home/jack/AnimalC/cars-srsa-open-data-animal-related-complaints-oct-to-dec-2019.csv")

In [24]:
jan_to_mar_20 = pd.read_csv("/home/jack/AnimalC/cars-srsa-open-data-animal-related-complaints-jan-to-mar-2020.csv")